In [13]:
import tensorflow as tf

import numpy as np
import os
import time

In [14]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [15]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [16]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



## Process the Text

### Vectorize the Text

In [17]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

#### StringLookup Layer

In [19]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

In [20]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[3, 0, 4, 5, 6, 0, 7], [19, 0, 0]]>

In [22]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [23]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'[UNK]', b'c', b'd', b'e', b'[UNK]', b'g'],
 [b'x', b'[UNK]', b'[UNK]']]>

In [24]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'a[UNK]cde[UNK]g', b'x[UNK][UNK]'], dtype=object)

In [25]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [26]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([ 0,  9, 15, ...,  7,  2,  0], shape=(1115394,))>

In [27]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [28]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

[UNK]
i
r
s
t
 
[UNK]
i
t
i


In [29]:
seq_length = 100

In [30]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'[UNK]' b'i' b'r' b's' b't' b' ' b'[UNK]' b'i' b't' b'i' b'[UNK]' b'e'
 b'n' b'[UNK]' b'[UNK]' b'[UNK]' b'e' b'[UNK]' b'o' b'r' b'e' b' ' b'w'
 b'e' b' ' b'p' b'r' b'o' b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'[UNK]' b' '
 b'[UNK]' b'[UNK]' b'r' b't' b'h' b'e' b'r' b'[UNK]' b' ' b'h' b'e' b'a'
 b'r' b' ' b'm' b'e' b' ' b's' b'p' b'e' b'a' b'[UNK]' b'.' b'[UNK]'
 b'[UNK]' b'[UNK]' b'l' b'l' b'[UNK]' b'[UNK]' b'[UNK]' b'p' b'e' b'a'
 b'[UNK]' b'[UNK]' b' ' b's' b'p' b'e' b'a' b'[UNK]' b'.' b'[UNK]'
 b'[UNK]' b'[UNK]' b'i' b'r' b's' b't' b' ' b'[UNK]' b'i' b't' b'i'
 b'[UNK]' b'e' b'n' b'[UNK]' b'[UNK]' b'[UNK]' b'o' b'[UNK]' b' '], shape=(101,), dtype=string)


In [31]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'[UNK]irst [UNK]iti[UNK]en[UNK][UNK][UNK]e[UNK]ore we proceed an[UNK] [UNK][UNK]rther[UNK] hear me spea[UNK].[UNK][UNK][UNK]ll[UNK][UNK][UNK]pea[UNK][UNK] spea[UNK].[UNK][UNK][UNK]irst [UNK]iti[UNK]en[UNK][UNK][UNK]o[UNK] '
b'are all resol[UNK]ed rather to die than to [UNK]amish[UNK][UNK][UNK][UNK]ll[UNK][UNK][UNK]esol[UNK]ed. resol[UNK]ed.[UNK][UNK][UNK]irst [UNK]iti[UNK]en[UNK][UNK][UNK]irst[UNK] [UNK]o[UNK] [UNK]'
b'now [UNK]ai[UNK]s [UNK]arci[UNK]s is chie[UNK] enem[UNK] to the people.[UNK][UNK][UNK]ll[UNK][UNK][UNK]e [UNK]now[UNK]t[UNK] we [UNK]now[UNK]t.[UNK][UNK][UNK]irst [UNK]iti[UNK]en[UNK][UNK][UNK]et [UNK]s [UNK]i'
b'll him[UNK] and we[UNK]ll ha[UNK]e corn at o[UNK]r own price.[UNK][UNK]s[UNK]t a [UNK]erdict[UNK][UNK][UNK][UNK]ll[UNK][UNK][UNK]o more tal[UNK]ing on[UNK]t[UNK] let it [UNK]e d'
b'one[UNK] awa[UNK][UNK] awa[UNK][UNK][UNK][UNK][UNK]econd [UNK]iti[UNK]en[UNK][UNK][UNK]ne word[UNK] good citi[UNK]ens.[UNK][UNK][UNK]irst [UNK]iti[UNK]en[UNK][UNK][UNK]e are acco[UNK

In [32]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [33]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [34]:
dataset = sequences.map(split_input_target)

In [35]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'[UNK]irst [UNK]iti[UNK]en[UNK][UNK][UNK]e[UNK]ore we proceed an[UNK] [UNK][UNK]rther[UNK] hear me spea[UNK].[UNK][UNK][UNK]ll[UNK][UNK][UNK]pea[UNK][UNK] spea[UNK].[UNK][UNK][UNK]irst [UNK]iti[UNK]en[UNK][UNK][UNK]o[UNK]'
Target: b'irst [UNK]iti[UNK]en[UNK][UNK][UNK]e[UNK]ore we proceed an[UNK] [UNK][UNK]rther[UNK] hear me spea[UNK].[UNK][UNK][UNK]ll[UNK][UNK][UNK]pea[UNK][UNK] spea[UNK].[UNK][UNK][UNK]irst [UNK]iti[UNK]en[UNK][UNK][UNK]o[UNK] '


In [36]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [37]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [38]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [43]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [44]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [45]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

InvalidArgumentError: Exception encountered when calling MyModel.call().

[1m{{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [64,100,256] != values[1].shape = [] [Op:Pack] name: [0m

Arguments received by MyModel.call():
  • inputs=tf.Tensor(shape=(64, 100), dtype=int64)
  • states=None
  • return_state=False
  • training=False